In [ ]:
import chardet
import pandas as pd
import numpy as np

# Tratamiento de datos: Muertes de Polución

Este cuaderno se dedica a preparar el conjunto de datos **Muerte por Polución** para adaptarlo correctamente a cómo podría entenderlo un algoritmo de Aprendizaje Automático. Esto es importante, porque actualmente, la estructura de datos está pensada para la visualización de los datos y su comprensión por humanos.

Como aconsejan las buenas prácticas, la carga de las bibliotecas que se utilizarán en el proceso debe hacerse al principio. Sobre las bibliotecas que se cargas, a la hora de realizarlo, las bibliotecas se han utilizado en estas versiones:
- chardet 3.0.4
- pandas 1.2.0
- numpy 1.19.5

### Introducción
En la mayoría de casos, los datos con los que trabajamos no tienen una estructura adecuada para su gestión con un algoritmo de Aprendizaje Automático y tenemos que hacer una conversión de las tablas para ajustarlas de forma que podamos acceder y utilizar los datos de manera óptima.

Muchas veces, los conjuntos de datos llegan a nosotros preparados para generar visualizaciones. Necesitamos entender los datos de los que disponemos y manipular las tablas para generar una estructura de datos adecuada al problema que estemos tratando de solucionar; para ello, realizaremos transposiciones, agrupaciones y/o extracciones.

Hay que tener en cuenta la ética del proceso y entender que una manipulación de datos no tiene nada que ver con tergiversarlos o sesgarlos de alguna manera, sino que estamos planteando una transformación de su estructura.

Para aquellos que estén comenzando a realizar procesos de limpieza y tratamiento de datos con Python y Pandas a través de cuadernos Jupyter, tened presente que necesitáis reasignar las variables a los cambios que vayáis haciendo en el conjunto de datos. Es igualmente importante mantener la integridad del original para poder recuperar los datos de origen si en algún momento hicieran falta. Pandas, además, dispone de parámetros en algunas funciones como `drop` que permiten realizar cambios en el conjunto de datos sin tener que reasignar la variable, busca si las funciones que quieres utilizar lo permiten en la documentación de Pandas y luego actívalo añadiéndolo dentro del paréntesis: `inplace=True`.

### Carga del conjunto de datos

In [ ]:
ruta_archivo = './datos/airPollutionDeathRate.csv'

def pd_abrir_archivo(ruta_archivo, separador=','):
    with open(ruta_archivo, 'rb') as original:
        resultado = chardet.detect(original.read())
    return pd.read_csv(ruta_archivo, sep=separador, encoding=resultado['encoding'])

data = pd_abrir_archivo(ruta_archivo)
data.drop(['Period'], axis=1, inplace=True)
data.sample(10)

En este conjunto de datos existe una gran cantidad de columnas categóricas que generan una gran cantidad de ruido y no nos permiten acceder correctamente a la información que necesitamos. En este caso, queremos que `First Tooltip`, que es la columna que nos está aportando la información, se muestre con una clara referencia hacia el país al que está haciendo referencia (`Location`); para ello, debemos generar nuevas columnas diferenciadas a través de los valores almacenados actualmente en `Dim2`, `Indicator` y `Dim1`. Ahora, **¿cómo llevaríamos a cabo el proceso?**

Es recomendable, antes de comenzar, que nos hagamos una lista de los pasos a seguir para realizar:
1. Queremos trabajar con las filas, por lo que necesitamos realizar una **transposición del conjunto**.
2. Tenemos que extraer de alguna manera el nombre del país y convertirlo en el **índice** de la fila del que queremos mantener la información: `First Tooltip`.
3. Lo siguiente es generar un nombre identificativo para las columnas, podríamos hacerlo con `Dim2`, `Indicator` y `Dim1`.
4. Hay que eliminar el ruido, es decir, las columnas que ya no nos hacen falta.

Se podría considerar una buena práctica realizar este proceso sobre una muestra de los datos, viendo cómo se comporta a la hora de aplicar el proceso de transformación que estamos desarrollando. En este caso, podemos utilizar como muestra un único país y, cuando consigamos hacer la transformación del conjunto, automatizar el proceso a través de una función.

### Planteamiento de la solución paso a paso

1. Lo primero que necesitamos es realizar la transposición del conjunto, este nuevo conjunto lo guardaremos en la variable `data_T` a la que, además, limitaremos por la columna de `Location` a **Afganistán**. Esta muestra se puede realizar de varias maneras; podemos hacer un breve análisis para saber que existen 36 filas (0-35) por cada país, por lo que podemos aplicar un `iloc` para seleccionar ese rango.

In [ ]:
data_T = data.T.iloc[:,0:36]
data_T

2. Lo siguiente que queremos es recuperar el nombre del país para luego renombrar `First Tooltip` con él. La opción fácil en este caso sería poner manualmente en nombre del país, pero nos interesa realizarlo a través de una selección que poder automatizar posteriormente. Para que visualmente entendamos mejor el proceso, lo haremos por pasos pequeños:
    - Asignamos a una variable el lugar donde sabemos que en el conjunto va a estar el nombre; podemos hacerlo a través, por ejemplo, de .iloc[0,0].
    - Renombramos, con `.rename`, el índice de la fila, pasando a llamarse `First Tooltip` con el nombre que hemos almacenado en `pais`.
    - Una vez que hemos terminado y viendo que no necesitaremos la fila de `Location`, es recomendable eliminarla.

In [ ]:
pais = data_T.iloc[0,0]
data_T.rename(index={'First Tooltip':pais}, inplace=True)
data_T.drop(['Location'], axis=0, inplace=True)
data_T

Cuando hayamos hecho los cambios, ya tenemos en una fila los datos de **Afganistán**, ahora nos queda que las columnas tengan un nombre significativo.

3. Como los datos que identifican las columnas están actualmente almacenados en tres filas diferentes, podemos realizar este trabajo generando un vector en el que se almacenen concatenados los textos contenidos en estas tres filas; para ello:
    - Guardamos en una variable a la que podemos llamar, por ejemplo, `vector_txt` la concatenación de los textos.
    - Una vez tengamos este vector hecho, lo único que tenemos que hacer es lo mismo que ya hicimos con  `First Tooltip`, pero esta vez, la sustitución del nombre no afectará **índice**, sino a las **columnas**.
    - Finalmente, para mantener el conjunto que estamos convirtiendo lo más limpio posible, eliminamos las filas que ya no nos hacen falta (`Dim2`, `Indicator` y `Dim1`)

In [ ]:
vector_txt = data_T.iloc[0,:] + " | " + data_T.iloc[1,:] + " | " + data_T.iloc[2,:]
data_T.rename(columns=vector_txt, inplace=True)
data_T.drop(['Dim2', 'Indicator', 'Dim1'], axis=0, inplace=True)

### Automatización del proceso

Una vez tenemos todos los pasos para conseguir el resultado que buscamos, podemos generalizarlo y crear una o dos funciones que, al aplicarlas, nos faciliten la creación de un nuevo conjunto de datos que nos sirva para seguir trabajando y extrayendo información de él.

La creación de funciones, en este momento, nos sirve para ordenar y hacer el todo el proceso de forma cómoda; además, nos sirve para ver todo el desarrollo de un vistazo; por supuesto, la conversión de un proceso que hemos hecho manualmente sobre una muestra requiere de modificaciones y alteraciones para hacerla bien. Así, el proceso que hemos llevado hasta ahora, se simplificaría en la función `transformacion_df`.

In [ ]:
def transformacion_df(df_paises):
    df_pais = df_paises.rename(index={'First Tooltip':df_paises.iloc[0,0]})
    vector_txt = df_pais.iloc[1,:] + " | " + df_pais.iloc[2,:] + " | " + df_pais.iloc[3,:]
    df_pais = df_pais.rename(columns=vector_txt)
    df_pais = df_pais.drop(['Location', 'Dim2', 'Indicator', 'Dim1'], axis=0)
    return df_pais

#### Explicando la función `transformación_df` línea a línea

`def transformacion_df(df_paises):`

Introduciendo en la función como parámetro de entrada el conjunto ya transpuesto y filtrado, lo primero que hacemos es, según el valor que aparece en `df_paises.iloc[0,0]`, sustituir el nombre del índice del conjunto `df_pais` que antes se llamaba `First Tooltip` por el nombre del país.

`df_pais = df_paises.rename(index={'First Tooltip':df_paises.iloc[0,0]})`

Creamos una variable llamada `vector_txt` que contiene **un vector de información** que concatena los valores contenidos en las celdas específicas de `df_pais` que le hemos especificado a través de `.iloc`.

`vector_txt = df_pais.iloc[1,:] + " | " + df_pais.iloc[2,:] + " | " + df_pais.iloc[3,:]`

Utilizamos `vector_txt` para renombrar las columnas del conjunto `df_pais`

`df_pais = df_pais.rename(columns=vector_txt)`

Eliminamos las filas que contienen los datos que ya no vamos a necesitar a través de un drop: `Location`, `Dim2`, `Indicator`, `Dim1`

`df_pais = df_pais.drop(['Location', 'Dim2', 'Indicator', 'Dim1'], axis=0)`

Una vez finalizado el proceso, la función nos devuelve el conjunto de `df_pais` con las modificaciones hechas.

`return df_pais`

## Aplicación del proceso a todo el conjunto

Ahora nos toca aplicar la función de transormación a todo el conjunto que hemos cargado; para ello, crearemos un bucle `for` que nos extraiga del conjunto de datos original la lista de los países que contiene. Esta lista la podemos conseguir fácilmente extrayendo los valores únicos de la columna `Location` con `.unique()`.

Por supuesto, el bucle va a necesitar que hagamos dos cosas incialmente:
1. Que definamos un conjunto de datos vacío donde se almacenarán las nuevas filas que vayamos generando con la función de `transformacion_df`.
2. Que tengamos hecha ya la transposición del conjunto de datos original (`dfT`).

Una vez definidas estas dos variables, podemos enfrentarnos a la creación del bucle, **¿qué necesita hacer el bucle?** 
1. Necesita hacer la **selección** de los datos; como queremos seleccionar por país, tenemos que hacer un `.loc` que recupere todas las filas que sean del país determinado; para verlo claramente, esta selección la podemos guardar en una variable intermedia, la que hemos llamado `seleccion`.
2. A esta selección le aplicamos la función que acabamos de crear de `transformacion_df` que realizará todo el proceso con cada conjunto de pais que recupere de `df['Location']` y la almacenará en la variable `fila`.
3. Esta variable fila se irá concatenando con cada vuelta del bucle al conjunto final que hemos definido en `df_final`.

In [ ]:
df_final = pd.DataFrame()
data_T = data.T

for pais in data['Location'].unique():
    seleccion = data_T.loc[:, data_T.loc['Location'] == pais]
    fila = transformacion_df(seleccion)
    df_final = pd.concat([df_final, fila])

df_final

**La segunda función la generaremos con el bucle que hemos generado para trabajar el conjunto de datos**; si bien podemos decirle que el procesamiento lo haga en tramos del conjunto, tal como hicimos a la hora de recopilar las 36 primeras filas, esto nos podría suponer un problema si el conjunto añade alguna fila más de datos posteriormente.

Obviamente, estas dos funciones podrían plantearse fácilmente para que todo el proceso estuviese en una única función; el hecho de separarlas nos permite acceder a la de transformación sin necesitar la de conversión, lo que podría ser interesante si no queremos hacer todo el proceso sobre todo el conjunto.

In [ ]:
def convercion_completa_df(df):
    df_final = pd.DataFrame()
    dfT = df.T
    
    for pais in df['Location'].unique():
        seleccion = dfT.loc[:, dfT.loc['Location'] == pais]
        fila = transformacion_df(seleccion)
        df_final = pd.concat([df_final, fila])
    
    return df_final

df = convercion_completa_df(data).T
df

#### Explicando la función `convercion_completa_df` línea a línea

`def convercion_completa_df(df):`

Una vez cargado el conjunto de datos completo en la función, lo primero que se ejecuta es la creación de un conjunto de datos vacío que se almacena en `df_final` y se genera también un conjunto **transpuesto** del conjunto original que se almacena en `dfT`.

`df_final = pd.DataFrame()`


`dfT = df.T`

Creamos entonces un bucle **for** en el que para cada **país**, que esté en la lista que genera `.unique()` para los valores que estén en la columna `Location` del conjunto original `df`, será el que defina la selección posterior de las columnas que se utilizarán en el conjunto transpuesto `dfT`.

`for pais in df['Location'].unique():`

Seleccionamos, utilizando esta vez `.loc`, todas aquellas columnas que cumplan la condición de pertenecer al país que se está recuperando en ese momento y se almacena en `seleccion`.

`seleccion = dfT.loc[:, dfT.loc['Location'] == pais]`

A este conjunto filtrado que hemos almacenado en `seleccion` le aplicamos la función que hemos creado y definido **antes** de `transformacion_df` que tendrá como parámetro de entrada el conjunto filtrado y que se almacenará en la variable `fila`.

`fila = transformacion_df(seleccion)`

En el conjunto que hemos preparado para recibir los datos modificados, `df_final`, iremos en cada vuelta del bucle **concatenando** los resultados del proceso; una vez termine de ejecutarse, será este conjunto final el que nos devuelva la función.

`df_final = pd.concat([df_final, fila])`


`return df_final`

### Apuntes finales

Ten cuidado cuando definas una función que llama a otra y no te olvides **definir ambas** antes de llamar a la primera, ya que podría generarte un error al no encontrar alguna de las que necesita. Por ello, a la hora de definir las funciones, es recomendable que estén todas ordenadas **al inicio y en la misma celda** cuando trabajemos en Jupyter y así evitarnos errores inesperados.

En la siguiente celda, además, puedes ver cómo se han hecho algunos cambios a las funciones para permitir que sean aún más genéricas: 
* En `transformacion_df`, en vez de eliminar las filas específicas, lo que hacemos es seleccionar únicamente la fila que contiene en este caso el objetivo.
    - Además, como esta fila podría estar situada en otro orden, hacemos que uno de los parámetros de entrada sea `fila_objetivo`, que nos permitirá modificar el número de la columna en la que se encuentran los datos objetivo. (Digo columna porque el número a introducir sería el natural (no comenzando desde 0) de la posición de la columna objetivo según el conjunto original).
    - Igualmente, he puesto como parámetros de entrada la columna que usamos para seleccionar los `paises`, y la columna donde están los datos que queremos mantener como `objetivo`.
    


In [ ]:
def transformacion_df(df_paises, objetivo, fila_objetivo, pais):
    '''Función que permite, dado unn conjunto de datos traspuesto, renombrar la columna objetivo por el nombre
    del país, renombrar las columnas según los valores de las filas de características y eliminar las filas del
    conjunto que, tras la transformación, ya no son necesarias.'''
    
    df_pais = df_paises.rename(index={objetivo:pais})
    vector_txt = df_pais.iloc[1,:] + " | " + df_pais.iloc[2,:] + " | " + df_pais.iloc[3,:]
    df_pais = df_pais.rename(columns=vector_txt)
    df_pais = df_pais[fila_objetivo-1: fila_objetivo]
    
    return df_pais

def convercion_completa_df(df, paises, objetivo, fila_objetivo):
    '''Función que genera un conjunto de datos modificados a partir de un conjunto de datos determinado,
    seleccionando según los datos almacenados en la columna de datos seleccionada, renombrar el índice
    de la fila objetivo en el conjunto transpuesto.
    Para un funcionamiento correcto, requiere de haber definido la función transformacion_df'''
    
    df_final = pd.DataFrame()
    dfT = df.T
    for pais in df[paises].unique():
        seleccion = dfT.loc[:, dfT.loc[paises] == pais]
        fila = transformacion_df(seleccion, objetivo, fila_objetivo, pais)
        df_final = pd.concat([df_final, fila])
    
    return df_final

In [ ]:
convercion_completa_df(data, 'Location', 'First Tooltip', 5)

Ya tenemos transformados los datos. Ahora habría que separar los valores contenidos para que los reconozca como numéricos y poder acceder también a esos mínimos y a esos máximos. Pero eso ya es otra historia.

#### Origen de los datos

El conjunto original con el que se ha hecho este ejemplo se ha descargado de la web de la Organización Mundial de la Salud (https://www.who.int/health-topics/air-pollution)